In [8]:
from bson import json_util, ObjectId
from pandas.io.json import json_normalize
import json
import pymongo
import pandas as pd
from pymongo import MongoClient as Connection

connection = Connection()
db = connection.recipe_db
input_data = db.recipes_test_copy
#data = pd.DataFrame(list(input_data.find()))

data = json.loads(json_util.dumps(input_data.find()))
norm_data = json_normalize(data)

ingredients = json_normalize(data, record_path='ingredients', meta='id', record_prefix='ingredients_', errors='ignore' )
ingredients = pd.DataFrame(ingredients)

#schmeiss alle zutaten raus, die weniger als 4 mal verwendet werden
ingredients_unique = ingredients[ingredients.groupby('ingredients_id').ingredients_id.transform(len) > 4]

# drop duplicate by a column name
 
ingredients_unique.drop_duplicates(['ingredients_id'], keep='last')

ingredients_unique = ingredients_unique.drop(ingredients_unique.columns[[0, 3, 4]], axis=1)

#ingredients_unique['ingredients_id'].value_counts()


#schmeiss alle zutaten raus, die weniger als 4 mal verwendet werden
ingredients_trans = ingredients[ingredients.groupby('ingredients_id').ingredients_id.transform(len) > 4]

test2 = pd.get_dummies(ingredients_trans['ingredients_id']).groupby(ingredients_trans['id']).apply(max)

id_list_five = [236609, 65896, 9023, 11758, 24264]
id_list_one = [16354]
#de = test2.iloc[id_list]
#test id = 16354
#7025

test_df_one_recipe = test2[test2.index.isin(id_list_one)]

test_df_five_recipes = test2[test2.index.isin(id_list_five)]

test_df_five_recipes

,ingredients_grams,ingredients_id,ingredients_name,ingredients_type,id
0,11.360000,16157,10 g butter,Normal,59661
1,41.800000,4405,40 g sliced green onions,Normal,59661
2,4.800000,4342,"1-1/2 cloves garlic, minced",Normal,59661
3,227.200000,4520,3/4 (10 ounce) package frozen chopped spinach ...,Normal,59661
4,182.400009,16243,180 g ricotta cheese,Normal,59661


In [9]:

ingredients_trans.head()

,ingredients_grams,ingredients_id,ingredients_name,ingredients_type,id
0,11.360000,16157,10 g butter,Normal,59661
1,41.800000,4405,40 g sliced green onions,Normal,59661
2,4.800000,4342,"1-1/2 cloves garlic, minced",Normal,59661
3,227.200000,4520,3/4 (10 ounce) package frozen chopped spinach ...,Normal,59661
4,182.400009,16243,180 g ricotta cheese,Normal,59661


In [5]:
test_df_one_recipe

,0,111,126,257,388,443,445,578,615,629,...,24850,24865,25518,25522,26269,26706,26934,27343,27388,27805
id,,,,,,,,,,,,,,,,,,,,,
16354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from sklearn.metrics.pairwise import cosine_similarity



cosine_similarity(test_df_five_recipes, test_df_one_recipe)

array([[0.        ],
       [0.11785113],
       [0.55555556],
       [0.1490712 ],
       [0.13608276]])

In [4]:
cosine_similarity(test_df_one_recipe, test_df_five_recipes)

array([[0.        , 0.11785113, 0.55555556, 0.1490712 , 0.13608276]])

In [6]:
ary = scipy.spatial.distance.cdist(test_df_one_recipe.iloc[:,1:], test_df_five_recipes.iloc[:,1:], metric='euclidean')

pd.DataFrame(ary)

NameError: name 'scipy' is not defined